In [ ]:
import argparse
import json
import os
from time import time
from pathlib import Path

import joblib
import mlflow
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

pd.set_option("display.max_columns", None)

# Experiment Tracking

In [ ]:
DATA_DIR = Path("../data")
SPLIT_DATA_DIR = DATA_DIR / "split"
TRAIN_DATA_FILENAME = SPLIT_DATA_DIR / "lending_club_train.csv"
TEST_DATA_FILENAME = SPLIT_DATA_DIR / "lending_club_test.csv"

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_FILENAME)
df_test = pd.read_csv(TEST_DATA_FILENAME)

# columns to drop
col_to_drop = ["issue_d", "loan_status", "revol_util", "pub_rec_bankruptcies"]
X = df_train.drop(columns=col_to_drop)
y = df_train["loan_status"]

X_test = df_test.drop(columns=col_to_drop)
y_test = df_test["loan_status"]

print("Training size:", X.shape[0])
print("Test size:", X_test.shape[0])

list_features = X.select_dtypes(include="number").columns.tolist()
X_train = X[list_features].copy()
y_train = y.copy()

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""

In [ ]:
mlflow.set_tracking_uri("")

In [ ]:
mlflow.set_experiment("dvc-mlflow-test")

In [ ]:
mlflow.sklearn.autolog()

base_model = Pipeline(
    [
        ("estimator", RandomForestClassifier(random_state=11))
    ]
)
base_model.fit(X_train, y_train)
train_preds = base_model.predict(X_train)
test_preds = base_model.predict(X_test[list_features])

print("Training performance..")
print(classification_report(y_train, train_preds))
print()
print("Test performance..")
print(classification_report(y_test, test_preds))

mlflow.sklearn.autolog(disable=True)

In [ ]:
mlflow.sklearn.autolog()

HYPERPARAMS = {
    "estimator__n_estimators": [5, 10, 50],
    "estimator__max_depth": [5, 10]
}
pipeline = Pipeline(
    [
        ("estimator", RandomForestClassifier(random_state=11))
    ]
)
model = GridSearchCV(pipeline, param_grid=HYPERPARAMS, verbose=2)
model.fit(X_train, y_train)
train_preds = model.predict(X_train)
test_preds = model.predict(X_test[list_features])

print("Training performance..")
print(classification_report(y_train, train_preds))
print()
print("Test performance..")
print(classification_report(y_test, test_preds))

mlflow.sklearn.autolog(disable=True)

In [ ]:
model_v1 = mlflow.sklearn.load_model("models:/dvc-mlflow-models/1")

train_preds = model_v1.predict(X_train)
test_preds = model_v1.predict(X_test[list_features])

print("Training performance..")
print(classification_report(y_train, train_preds))
print()
print("Test performance..")
print(classification_report(y_test, test_preds))

In [ ]:
model_v2 = mlflow.sklearn.load_model("models:/dvc-mlflow-models/2")

train_preds = model_v2.predict(X_train)
test_preds = model_v2.predict(X_test[list_features])

print("Training performance..")
print(classification_report(y_train, train_preds))
print()
print("Test performance..")
print(classification_report(y_test, test_preds))